# Expectation-Maximization (EM) for parameter inference and hyper-parameter optimization

### Defining the likelihood 

A central goal of modeling is to predict the distribution of possible outcomes that could occur under a given experimental condition. A data driven approach for developing a model is to use previously collected experimental data to train a model to perform this task. Measurements from individual experimental conditions are almost always assumed to be independent, meaning that observing the outcomes from an experimental condition will not influence the outcome of an other experimental condition. Experimental measurements are imprecise due to measurement noise, and because there are typically many sources of noise, the central limit theorem justifies the assumption that measurement noise will be normally distributed. The choice of distribution to characterize noise is called the *noise model*. We use a different model, sometimes called the *central model*, to predict the expected value of an outcome from a particular experimental condition and characterize the noise using the noise model, 

\begin{equation}
    y(q_i) = f(q_i, \theta) + \varepsilon
    \label{eq: model}
\end{equation}
where $q_i$ denotes a particular experimental condition, $y(q_i)$ denotes the measurement from the experimental condition,  $f(q_i, \theta)$ is the model prediction of the measurement, $\theta$ is a vector of model parameters, and $\varepsilon \sim \mathcal{N}(0, \sigma^2)$ is a random variable sampled from the noise model. The central model and noise model together define the sampling distribution,

\begin{equation}
    p(y(q_i) | q_i, \theta, \sigma^2) = \mathcal{N}(y(q_i) | f(q_i, \theta), \sigma^2),
    \label{eq: sampling distribution}
\end{equation}
which quantifies how likely a measurement was sampled from the predicted conditional distribution given the experimental condition, $q_i$. 
An experimental design is a set of experimental conditions, $\mathbf{q} = \{q_1, ..., q_n \}$, and data collected from an experimental design is denoted as $\mathcal{D}(\mathbf{q}) = \{y(q_1), ..., y(q_n) \}$. Because measurements from each experimental condition are assumed to be drawn independently from the sampling distribution, the likelihood of a set of measurements is

\begin{equation}
    p(\mathcal{D}(\mathbf{q}) | \mathbf{q}, \theta, \sigma^2) = \prod_{i=1}^{n} \mathcal{N}(y(q_i) | f(q_i, \theta), \sigma^2).
\end{equation}
One approach for parameter estimation is to maximize the log of the likelihood function with respect to parameters, 

\begin{align}
    \theta_{MLE} &= \underset{\theta}{\text{argmax}} \; \text{log} \; p(\mathcal{D}(\mathbf{q}) | \theta, \sigma^2) \nonumber \\ &= \underset{\theta}{\text{argmax}} \; \sum_{i=1}^n \left( - \frac{(y(q_i) - f(q_i, \theta))^2}{2\sigma^2} - \text{log} \; \sigma \right).
\end{align}
A limitation of the maximum likelihood approach is that it does not allow for the incorporation of prior knowledge about the parameters, which in the simplest case can be useful for promoting parsimonious parameter estimates by setting the mean of the parameter prior to zero. Furthermore, incorporating a prior enables a Bayesian approach to infer a posterior parameter distribution, which is useful for understanding how well parameters are constrained by data, quantifying model prediction uncertainty, model selection, and designing informative experiments that can be used to further refine the model. 

### Variational Inference and Expectation-Maximization for parameter inference and hyper-parameter optimization

The goal of Bayesian parameter inference is to compute a posterior parameter distribution conditioned on all available data, denoted as $p(\theta | \mathcal{D}(\mathbf{q}))$. Computing a posterior distribution analytically is only possible for models that are linear with respect to parameters; however, posterior distributions can be approximated by either sampling from the distribution using methods like Markov chain Monte Carlo (MCMC) or by optimizing the parameters, denoted as $\phi$, of an approximate probability density function, $z(\theta | \phi) \approx p(\theta | \mathcal{D}(\mathbf{q}))$. We will use an independent Gaussian prior distribution for each parameter, $p(\theta_k | \alpha_k) = \mathcal{N}(\theta_k | 0, 1/ \alpha_k)$, where $\alpha_k$ is the precision (inverse variance) of $\theta_k$.    
Measurement noise is also assumed to be Gaussian with variance, $\sigma^2$. The set of variables, $\xi = \{\alpha_1, ..., \alpha_{n_\theta}, \sigma^{2} \}$, are considered hyper-parameters, because they influence parameter inference but are distinct from the model parameters, $\theta$. The objective function that takes into account model parameters and hyper-parameters is the log marginal likelihood given by

\begin{equation}
\text{log} \; p (\mathcal{D}(\mathbf{q}) | \xi) = \text{log} \; \int_{\theta} p (\mathcal{D}(\mathbf{q}), \theta | \xi)  d \theta.
\end{equation}
For any choice of $z(\theta | \phi)$, the log marginal likelihood can be decomposed into two functions, 

\begin{equation}
    \text{log} \; p (\mathcal{D}(\mathbf{q}) | \xi) = 
    \underbrace{\int_{\theta} \text{log} \left( \frac{p(\mathcal{D}(\mathbf{q}), \theta | \xi)}{z(\theta | \phi)} \right) z(\theta | \phi) d \theta}_{\mathcal{L}(z(\theta | \phi), \xi)} + 
    \underbrace{\int_{\theta} -\text{log} \left( \frac{p(\theta | \mathcal{D}(\mathbf{q}), \xi)}{z(\theta | \phi)} \right) z(\theta | \phi) d \theta}_{\text{KL}}.
\end{equation}
where $\text{KL}$ is the Kullback-Leibler divergence between the approximating distribution $z(\theta | \phi)$ and the true posterior parameter distribution $p(\theta | \mathcal{D}(\mathbf{q}), \xi)$. Because the KL divergence is strictly non-negative, the function $\mathcal{L}(z(\theta | \phi), \xi)$ is a lower bound on the log marginal likelihood since $\mathcal{L}(z(\theta | \phi), \xi) \leq \text{log} \; p (\mathcal{D}(\mathbf{q}) | \xi)$. This lower bound is referred to as the evidence lower bound (ELBO). Optimizing the ELBO with respect to the parameters of the approximate posterior distribution, $\phi$, is called variational inference. The Expectation-Maximization algorithm iterates between an expectation step, which involves variational inference to maximize the ELBO with respect to $\phi$ keeping $\xi$ fixed, followed by a maximization step, in which the ELBO is maximized with respect to $\xi$ keeping $\phi$ fixed.

### Expectation step

The expectiation step involves first optimizing the parameters of the approximate posterior distribution, $\phi$, 

\begin{equation}
    \phi^* = \underset{\phi}{\text{argmax}} \; \mathcal{L}(z(\theta | \phi), \xi)
\end{equation}
and then evaluating the expectation using the optimized parameters, 

\begin{align}
    \mathcal{L}(z(\theta | \phi^*), \xi) &= \int_{\theta} \text{log} \left( \frac{p(\mathcal{D}(\mathbf{q}), \theta | \xi)}{z(\theta | \phi^*)} \right) z(\theta | \phi^*) d \theta \nonumber \\ 
    &= \mathbb{E}_{z | \phi^*} \left[ \text{log} \left( \frac{p(\mathcal{D}(\mathbf{q}), \theta | \xi)}{z(\theta | \phi^*)} \right) \right]
    \label{eq: ELBO}
\end{align}
Using a Gaussian approximation of the posterior parameter distribution, the variational parameters are the mean and covariance, $z(\theta | \phi) = \mathcal{N}(\theta | \mu, \mathbf{\Sigma})$. Evaluating the ELBO (keeping only terms that depend on the variational parameters) gives, 

\begin{align}
    \mathcal{L}(\mathcal{N}(\theta | \mu, \mathbf{\Sigma}), \xi) &= \int_{\theta} \text{log} \left( \frac{p(\mathcal{D}(\mathbf{q}), \theta | \xi)} {\mathcal{N}(\theta | \mu, \mathbf{\Sigma})} \right) \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta \nonumber \\ 
    &= \int_{\theta} \text{log} \left( p(\mathcal{D}(\mathbf{q}) | \theta, \sigma^2) \right) \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta \nonumber \\  
    &\quad + \int_{\theta} \text{log} \left( \mathcal{N}(\theta | \mathbf{0}, \text{diag}(1/\mathbf{\alpha})) \right) \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta \nonumber \\ 
    &\quad - \int_{\theta} \text{log} \left( \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) \right) \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta \nonumber \\ 
    &= \int_{\theta} \sum_{i=1}^n \left( - \frac{(y(q_i) - f(q_i, \theta))^2}{2 \sigma^2} - \text{log} \; \sigma \right) \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta \nonumber \\
    &\quad + \sum_{k=1}^{n_{\theta}} \left( -\frac{1}{2} \alpha_k \mu_k^2 -\frac{1}{2} \alpha_k \Sigma_{kk} + \frac{1}{2} \text{log} \; \alpha_k \right) \nonumber \\ 
    &\quad + \frac{1}{2} \; \text{log} \; \text{det} \; \mathbf{\Sigma} 
\end{align}

For a linear regression model, the parameters are the regression coefficients that map a basis function of the input to the output, 

\begin{equation}
f(q_i, \theta) := \theta ^T \cdot \phi(q_i) = \sum_{j=1}^{m} \theta_j \cdot \phi_j(q_i)
\end{equation}
where $\phi(q_i) \in \mathbb{R}^m$ is a basis function that operates on the inputs.

Evaluating the expectation with the linear model gives, 

\begin{align}
     &\int_{\theta} \sum_{i=1}^n \left( - \frac{(y(q_i) - \theta^T \cdot \phi(q_i))^2}{2 \sigma^2} - \text{log} \; \sigma \right) \mathcal{N}(\theta | \mu,   \mathbf{\Sigma}) d \theta \nonumber \\
\end{align}

\begin{align}
     &= \sum_{i=1}^n \left( -\frac{1}{2 \sigma^2} \int_{\theta} (y(q_i) - \theta^T \cdot \phi(q_i))^2 \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta - \text{log} \; \sigma \right) \nonumber \\ 
\end{align}

\begin{align}
     &= -\frac{1}{2 \sigma^2} \sum_{i=1}^n \left( \int_{\theta} (y(q_i) - \theta^T \cdot \phi(q_i))^2 \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta \right)  - n \; \text{log} \; \sigma \nonumber \\ 
\end{align}

\begin{align}
     &= -\frac{1}{2 \sigma^2} \sum_{i=1}^n \left( y^2(q_i) -2 y(q_i) \mu^T \cdot \phi(q_i) + \int_{\theta} (\theta^T \phi(q_i))^2 \mathcal{N}(\theta | \mu, \mathbf{\Sigma}) d \theta \right)  - n \; \text{log} \; \sigma \nonumber \\ 
\end{align}

\begin{align}
     &= -\frac{1}{2 \sigma^2} \sum_{i=1}^n \left( y^2(q_i) -2 y(q_i) \mu^T \cdot \phi(q_i) + (\mu^T \phi(q_i))^2 + \phi(q_i)^T \Sigma \phi(q_i) \right)  - n \; \text{log} \; \sigma \nonumber \\ 
\end{align}

\begin{align}
     &= -\frac{1}{2 \sigma^2} \sum_{i=1}^n \left( (y(q_i) - \mu^T \cdot \phi(q_i) )^2 + \phi(q_i)^T \Sigma \phi(q_i) \right)  - n \; \text{log} \; \sigma \nonumber \\ 
\end{align}

Putting all of the terms together gives, 

\begin{align}
    \mathcal{L}(\mathcal{N}(\theta | \mu, \mathbf{\Sigma}), \xi) &= -\frac{1}{2 \sigma^2} \sum_{i=1}^n \left( (y(q_i) - \mu^T \cdot \phi(q_i) )^2 + \phi(q_i)^T \Sigma \phi(q_i) \right)  - n \; \text{log} \; \sigma \nonumber \\ 
    &\quad + \sum_{k=1}^{n_{\theta}} \left( -\frac{1}{2} \alpha_k \mu_k^2 -\frac{1}{2} \alpha_k \Sigma_{kk} + \frac{1}{2} \text{log} \; \alpha_k \right) \nonumber \\ 
    &\quad + \frac{1}{2} \; \text{log} \; \text{det} \; \mathbf{\Sigma}
\label{eq: approximate ELBO}
\end{align}

Maximizing the ELBO with respect to $\mathbf{\Sigma}$ gives, 

\begin{equation}
    \mathbf{\Sigma}^* = \left( \mathrm{diag}(\alpha) + \frac{1}{\sigma^2} \Phi^T \Phi \right)^{-1}
\end{equation}

Maximizing the expression for the ELBO with respect to $\mu$ gives, 

\begin{equation}
    \mu^* = \frac{1}{\sigma^2} \mathbf{\Sigma}^* \Phi^T \mathbf{y}
\end{equation}
where $\Phi_{ij} = \phi_j(q_i)$ and $\mathbf{y} = (y_1, ..., y_n)^T$. 

### Maximization step

The maximization step involves maximizing the ELBO where $\mu = \mu^*$ and $\mathbf{\Sigma} = \mathbf{\Sigma}^*$ with respect to each $\alpha_k$ and $\sigma^2$, which gives

\begin{equation}
    \alpha_k^* = \frac{1}{\mu_k^{*2} + \Sigma_{kk}^*}
\end{equation}
and

\begin{equation}
    \sigma^{*2} = \frac{1}{n} \sum_{i=1}^n (y(q_i) - \mu^{*T} \phi(q_i))^2 + \phi(q_i)^T \mathbf{\Sigma}^* \phi(q_i)
\end{equation}